In [1]:
!pip install ibm-watsonx-ai langchain langchain-ibm langchain-community gradio chromadb pypdf



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import warnings
from types import SimpleNamespace

# --- Suppress noisy warnings for a clean UI (optional) ---
def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings("ignore")

In [3]:
# --- LangChain & IBM watsonx ---
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings

In [4]:
# --- UI ---
import gradio as gr


# =========================
# LLM: watsonx Granite 3
# =========================
def get_llm():
    """
    Returns a WatsonxLLM instance (Granite 3 8B Instruct).
    Reads credentials from env vars:
        WATSONX_APIKEY, WATSONX_URL, WATSONX_PROJECT_ID
    """
    model_id = "ibm/granite-3-8b-instruct"
    params = {
        "temperature": 0.5,
        "max_new_tokens": 256,
        "decoding_method": "sample",
    }

    apikey = os.getenv("WATSONX_APIKEY")
    url = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")
    project_id = os.getenv("WATSONX_PROJECT_ID", "skills-network")

    if not apikey:
        raise RuntimeError(
            "Missing WATSONX_APIKEY. Set it as an environment variable."
        )

    llm = WatsonxLLM(
        model_id=model_id,
        url=url,
        apikey=apikey,
        project_id=project_id,
        params=params,
    )
    return llm



In [5]:
# ==================================
# 1) Document Loader (PyPDFLoader)
#    📸 Take screenshot: pdf_loader.png
# ==================================
def document_loader(file_path: str):
    """
    Loads a PDF and returns LangChain Documents list.
    Accepts a filesystem path string (Gradio provides this when type='filepath').
    """
    loader = PyPDFLoader(file_path)
    loaded_docs = loader.load()
    return loaded_docs

In [6]:
# ========================================
# 2) Text Splitter (RecursiveCharacter)
#    📸 Take screenshot: code_splitter.png
# ========================================
def text_splitter(documents):
    """
    Splits docs into manageable chunks.
    chunk_size/overlap tuned for RAG with small context windows.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = splitter.split_documents(documents)
    return chunks


In [7]:
# ===========================================
# 3) Embeddings (WatsonxEmbeddings)
#    📸 Take screenshot: embedding.png
# ===========================================
def watsonx_embedding():
    """
    Returns a WatsonxEmbeddings model for semantic vectors.
    Uses a small, fast retriever model (slate-125m).
    """
    apikey = os.getenv("WATSONX_APIKEY")
    url = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")
    project_id = os.getenv("WATSONX_PROJECT_ID", "skills-network")

    if not apikey:
        raise RuntimeError(
            "Missing WATSONX_APIKEY. Set it as an environment variable."
        )

    embed_params = {
        "truncation": True,
    }
    embedding = WatsonxEmbeddings(
        model_id="ibm/slate-125m-english-rtrvr",
        url=url,
        apikey=apikey,
        project_id=project_id,
        params=embed_params,
    )
    return embedding


In [8]:
# ====================================================
# 4) Vector DB (Chroma from_documents)
#    📸 Take screenshot: vectordb.png
# ====================================================
def vector_database(chunks):
    """
    Creates an in-memory Chroma vector store from chunks using watsonx embeddings.
    (No persistence; add persist_directory=... to persist between runs.)
    """
    embedding_model = watsonx_embedding()
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_model,
        # persist_directory="chroma_store"  # optional: uncomment to persist
    )
    return vectordb


In [9]:
# =========================================
# 5) Retriever (similarity search)
#    📸 Take screenshot: retriever.pnga
# =========================================
def retriever(file_path: str):
    """
    Full pipeline:
      load -> split -> embed -> build vector DB -> return retriever
    """
    docs = document_loader(file_path)
    chunks = text_splitter(docs)
    vectordb = vector_database(chunks)
    retr = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4})
    return retr



In [10]:
# ==========================================================
# 6) RetrievalQA Chain + Gradio Interface (the QA Bot)
#    📸 Take screenshot in the browser: QA_bot.png
#       - Interface visible
#       - PDF uploaded
#       - Query = "What this paper is talking about?"
# ==========================================================
def retriever_qa(file_path: str, query: str) -> str:
    """
    Build a RetrievalQA chain using:
      - LLM: Granite 3 8B Instruct
      - Retriever: Chroma similarity over PDF chunks
    Returns only the final answer string.
    """
    if not file_path:
        return "Please upload a PDF first."

    llm = get_llm()
    retr_obj = retriever(file_path)

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retr_obj,
        return_source_documents=False,  # set True if you want to inspect sources
    )
    # LCEL .invoke expects {"query": "..."} for RetrievalQA
    result = qa.invoke({"query": query or "What this paper is talking about?"})
    # Depending on LC version, result may be dict or str; handle both safely:
    if isinstance(result, dict) and "result" in result:
        return result["result"]
    if isinstance(result, str):
        return result
    return str(result)



In [13]:
# ======================
# Gradio App
# ======================
app_title = "Document Q&A Bot"
app_desc = (
    "Upload a PDF and ask any question. "
    "The bot will answer using the document via RAG (LangChain + watsonx)."
)

demo = gr.Interface(
    fn=lambda pdf_path, query: retriever_qa(pdf_path, query),
    inputs=[
        gr.File(
            label="Upload PDF File",
            file_count="single",
            file_types=[".pdf"],
            type="filepath",  # IMPORTANT: we pass a path string to our functions
        ),
        gr.Textbox(
            label="Input Query",
            lines=2,
            placeholder='Type your question here… e.g., "What this paper is talking about?"',
            value="What this paper is talking about?",
        ),
    ],
    outputs=gr.Textbox(label="Answer"),
    title=app_title,
    description=app_desc,
    allow_flagging="never",
)

In [16]:
if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7861)


Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\H

In [17]:
!taskkill /PID 12345 /F


ERROR: The process "12345" not found.
Traceback (most recent call last):
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Harsh Modi\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^